In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from neuralprophet import NBeats, LSTM, DeepAR, NeuralProphet, TFT
import pandas as pd
import numpy as np
import os

In [3]:
data_loc = '../../LIBRA/'
datasets = os.listdir(data_loc)

datasets_economics = [dataset for dataset in datasets if 'economics' in dataset]
datasets_finance = [dataset for dataset in datasets if 'finance' in dataset]
datasets_human = [dataset for dataset in datasets if 'human' in dataset]
datasets_nature = [dataset for dataset in datasets if 'nature' in dataset]

one-step-ahead, multistep, rolling_origin


In [4]:
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse

def smape(y_true, y_pred):
    return 200*np.mean(np.abs((y_true-y_pred))/(np.abs(y_true+y_pred)))
def mase(y_true, y_pred, y_pred_naive):
    
    naive_mae = mae(y_true, y_pred_naive)
    y_pred_mae = mae(y_true, y_pred)

    return y_pred_mae/naive_mae

def mues(y_true, y_pred):
    return(np.mean(np.maximum(np.sign(y_true-y_pred),0)))

def moes(y_true, y_pred):
    return(np.mean(np.maximum(np.sign(y_pred-y_true),0)))

def muas(y_true, y_pred):
    m = mues(y_true, y_pred)
    if m == 0:
        return 0
    else:
        return (1/m) * np.mean(np.maximum(y_true - y_pred, 0)/np.abs(y_true))

def moas(y_true, y_pred):
    m = moes(y_true, y_pred)
    if m == 0:
        return 0
    else:
        return (1/m) * np.mean(np.maximum(y_pred - y_true, 0)/np.abs(y_true))



In [21]:
usecase = 'finance'

datasets = {}

if usecase == 'economics':
    for dataset in datasets_economics:
        datasets.update({dataset:pd.read_csv(data_loc + dataset, header = None)})
elif usecase == 'finance':
    for dataset in datasets_finance:
        datasets.update({dataset:pd.read_csv(data_loc + dataset, header = None)})
elif usecase == 'human':
    for dataset in datasets_human:
        datasets.update({dataset:pd.read_csv(data_loc + dataset, header = None)})
elif usecase == 'nature':
    for dataset in datasets_nature:
        datasets.update({dataset:pd.read_csv(data_loc + dataset, header = None)})
    
    


In [22]:
datasets

{'finance_33.csv':                0
 0    2854.123802
 1    2862.026398
 2    2862.026398
 3    2877.831590
 4    2885.734186
 ..           ...
 319  1676.636996
 320  1668.734400
 321  1660.831804
 322  1652.929208
 323  1668.734400
 
 [324 rows x 1 columns],
 'finance_27.csv':                0
 0     111.314362
 1     111.681110
 2     109.629995
 3     109.049671
 4     111.845929
 ...          ...
 4206  557.732206
 4207  553.035318
 4208  553.635142
 4209  556.589981
 4210  554.253337
 
 [4211 rows x 1 columns],
 'finance_26.csv':                 0
 0     3694.299157
 1     3757.561279
 2     3846.590277
 3     3810.872056
 4     3762.803506
 ...           ...
 4205  3640.988380
 4206  3723.086977
 4207  3693.410644
 4208  3789.547746
 4209  3781.462278
 
 [4210 rows x 1 columns],
 'finance_32.csv':                0
 0     994.064991
 1     994.064991
 2    1008.736841
 3    1016.072766
 4    1016.072766
 ..           ...
 319  1815.688566
 320  1808.352641
 321  1830.360416
 322 

In [23]:
frequencies = pd.read_csv(data_loc + 'freq.csv')

In [24]:
frequencies[[col for col in frequencies.columns if usecase in col][0]].unique()

array([   1,    4,   12,    7,  360,  672,  120,   60, 1440, 2016,  288,
         96, 1320, 1800,   48,  180,  336, 4368,   84, 2880,   28,  730,
         30,  365,  245,  480,  240,  720])

In [27]:
mapping_frequencies_economics = {
    1 : 'D',
    4: 'Q',
    12: 'M',
    52: 'W',
    24: 'H',
    7: 'W',
    91: 'D',
    364: 'D', 
    360: 'D',
    168: 'D',
    672: 'D',
    96: 'H', 
    288: 'D',
    28: 'D',
    6: 'D',
    30: 'D',
    720: 'D'
    
}

def mapping(x):
    try:
        return mapping_frequencies_economics[x]
    except:
        return 'D'

In [28]:
for dataset, df in datasets.items():
    idx_ts = int(dataset.split('.')[0].split('_')[-1])-1
    freq = mapping(frequencies.iloc[
        idx_ts][[col for col in frequencies.columns if usecase in col][0]])
    df_ts = df.copy(deep = True)
    len_ts = df_ts.shape[0]
    df_ts['ds'] = pd.date_range(start = '2000-01-01', periods = len_ts, freq = freq)
    df_ts.columns = ['y', 'ds']
    datasets.update({dataset: df_ts})
    df_ts.to_csv(f'../../LIBRA_updated/{dataset}', index = False)

M
               y         ds
0    2854.123802 2000-01-31
1    2862.026398 2000-02-29
2    2862.026398 2000-03-31
3    2877.831590 2000-04-30
4    2885.734186 2000-05-31
..           ...        ...
319  1676.636996 2026-08-31
320  1668.734400 2026-09-30
321  1660.831804 2026-10-31
322  1652.929208 2026-11-30
323  1668.734400 2026-12-31

[324 rows x 2 columns]
D
               y         ds
0     111.314362 2000-01-01
1     111.681110 2000-01-02
2     109.629995 2000-01-03
3     109.049671 2000-01-04
4     111.845929 2000-01-05
...          ...        ...
4206  557.732206 2011-07-08
4207  553.035318 2011-07-09
4208  553.635142 2011-07-10
4209  556.589981 2011-07-11
4210  554.253337 2011-07-12

[4211 rows x 2 columns]
D
                y         ds
0     3694.299157 2000-01-01
1     3757.561279 2000-01-02
2     3846.590277 2000-01-03
3     3810.872056 2000-01-04
4     3762.803506 2000-01-05
...           ...        ...
4205  3640.988380 2011-07-07
4206  3723.086977 2011-07-08
4207  3693.4

W
                 0         ds
0      1222.723531 2000-01-02
1      1425.355651 2000-01-09
2      1296.603268 2000-01-16
3      1271.646131 2000-01-23
4      1240.738948 2000-01-30
...            ...        ...
2187  11818.433099 2041-12-01
2188  12466.492268 2041-12-08
2189  12466.492268 2041-12-15
2190  12604.665557 2041-12-22
2191  12604.665557 2041-12-29

[2192 rows x 2 columns]
W
                 0         ds
0      2861.064688 2000-01-02
1      2795.138658 2000-01-09
2      2930.008303 2000-01-16
3      2823.856221 2000-01-23
4      2981.553457 2000-01-30
...            ...        ...
1652  15674.684795 2031-08-31
1653  15565.534066 2031-09-07
1654  15448.239644 2031-09-14
1655  15536.305154 2031-09-21
1656  15685.479922 2031-09-28

[1657 rows x 2 columns]
D
              0         ds
0     24.268246 2000-01-01
1     24.021774 2000-01-02
2     24.310197 2000-01-03
3     24.179114 2000-01-04
4     24.336454 2000-01-05
...         ...        ...
5421  43.607446 2014-11-04
5422  43

D
               0         ds
0      47.754915 2000-01-01
1      47.719082 2000-01-02
2      47.667115 2000-01-03
3      47.995020 2000-01-04
4      47.745957 2000-01-05
...          ...        ...
5447  182.661072 2014-11-30
5448  184.882926 2014-12-01
5449  189.022040 2014-12-02
5450  186.692668 2014-12-03
5451  183.341959 2014-12-04

[5452 rows x 2 columns]
W
               0         ds
0    2047.954173 2000-01-02
1    2070.955902 2000-01-09
2    2056.647409 2000-01-16
3    2040.542950 2000-01-23
4    2030.183670 2000-01-30
..           ...        ...
652  3360.223954 2012-07-01
653  3373.867558 2012-07-08
654  3352.753502 2012-07-15
655  3356.935809 2012-07-22
656  3363.468155 2012-07-29

[657 rows x 2 columns]
D
              0         ds
0     42.336075 2000-01-01
1     43.069142 2000-01-02
2     38.273304 2000-01-03
3     41.064252 2000-01-04
4     38.741405 2000-01-05
...         ...        ...
1056  23.806302 2002-11-22
1057  22.967250 2002-11-23
1058  22.384331 2002-11-24
105

D
             0         ds
0    26.817482 2000-01-01
1    26.868331 2000-01-02
2    26.754428 2000-01-03
3    27.173432 2000-01-04
4    27.397171 2000-01-05
..         ...        ...
815  35.258560 2002-03-26
816  35.441621 2002-03-27
817  35.543318 2002-03-28
818  35.838249 2002-03-29
819  35.685701 2002-03-30

[820 rows x 2 columns]
D
                0         ds
0       39.592101 2000-01-01
1       39.342762 2000-01-02
2       39.405096 2000-01-03
3       38.781749 2000-01-04
4       39.093423 2000-01-05
...           ...        ...
12993  844.801602 2035-07-29
12994  848.572856 2035-07-30
12995  847.544332 2035-07-31
12996  849.383208 2035-08-01
12997  857.736069 2035-08-02

[12998 rows x 2 columns]
D
                0         ds
0        7.690361 2000-01-01
1        7.692155 2000-01-02
2        7.691258 2000-01-03
3        7.690361 2000-01-04
4        7.689463 2000-01-05
...           ...        ...
11451  199.590410 2031-05-09
11452  201.042022 2031-05-10
11453  202.989745 2031-

In [29]:
df_ts

,y,ds
0,2174.560252,2000-01-01
1,2289.969935,2000-01-02
2,2484.128006,2000-01-03
3,2736.266141,2000-01-04
4,3015.576969,2000-01-05
5,3148.198937,2000-01-06
6,3386.121646,2000-01-07
7,3513.265389,2000-01-08
8,3839.337714,2000-01-09
9,3910.904124,2000-01-10


In [30]:
datasets

{'finance_33.csv':                y         ds
 0    2854.123802 2000-01-31
 1    2862.026398 2000-02-29
 2    2862.026398 2000-03-31
 3    2877.831590 2000-04-30
 4    2885.734186 2000-05-31
 ..           ...        ...
 319  1676.636996 2026-08-31
 320  1668.734400 2026-09-30
 321  1660.831804 2026-10-31
 322  1652.929208 2026-11-30
 323  1668.734400 2026-12-31
 
 [324 rows x 2 columns],
 'finance_27.csv':                y         ds
 0     111.314362 2000-01-01
 1     111.681110 2000-01-02
 2     109.629995 2000-01-03
 3     109.049671 2000-01-04
 4     111.845929 2000-01-05
 ...          ...        ...
 4206  557.732206 2011-07-08
 4207  553.035318 2011-07-09
 4208  553.635142 2011-07-10
 4209  556.589981 2011-07-11
 4210  554.253337 2011-07-12
 
 [4211 rows x 2 columns],
 'finance_26.csv':                 y         ds
 0     3694.299157 2000-01-01
 1     3757.561279 2000-01-02
 2     3846.590277 2000-01-03
 3     3810.872056 2000-01-04
 4     3762.803506 2000-01-05
 ...           

In [31]:
valid_p = 0.2

In [33]:
from neuralprophet.utils.df_utils import split_df

metrics = {}

counter = 0
for dataset_name, df in datasets.items():
#     if counter == 1:
#         break
    
    idx_ts = int(dataset_name.split('.')[0].split('_')[-1])-1
    freq_number = frequencies.iloc[
        idx_ts][[col for col in frequencies.columns if usecase in col][0]]

    freq = mapping(frequencies.iloc[
        idx_ts][[col for col in frequencies.columns if usecase in col][0]])
    
    n_lags = freq_number
    n_forecasts = 3
    
    m = LSTM(n_lags = n_lags,
            n_forecasts=n_forecasts,
            num_hidden_layers=1,
            d_hidden=64,
            learning_rate=0.1,
            epochs=50,
            batch_size=None,
            loss_func="Huber",
            optimizer="AdamW",
            train_speed=None,
            normalize="auto",
            impute_missing=True,
            lstm_bias = True,
            lstm_bidirectional = False)
    
    tr, vl = split_df(df, n_lags = n_lags, n_forecasts = n_forecasts, valid_p = valid_p)
    m.fit(tr, freq = freq)
    future = m.make_future_dataframe(vl, periods = 0, n_historic_predictions=True)
    forecast = m.predict(future)
    fold = forecast.iloc[n_lags:][[f'yhat{i}' for i in range(1, n_forecasts+1)]]

    y_predicted = [np.array(fold).diagonal(offset=-i) for i in range(len(fold) - n_forecasts + 1)]
    y = np.array(vl[n_lags:]["y"])
    y_rolled = [y[i : i + n_forecasts] for i in range(len(y) - n_forecasts + 1)]
    
    y_naive = np.array(vl[n_lags-1:-1]["y"])
    y_naive_rolled = [y_naive[i : i + n_forecasts] for i in range(len(y_naive) - n_forecasts + 1)]
    
    smapes = np.mean([smape(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    mases = np.mean([mase(y_rolled[i], y_predicted[i], y_naive_rolled[i]) for i in range(len(y_rolled))])
    mueses = np.mean([mues(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    moeses = np.mean([moes(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    muases = np.mean([muas(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    moases = np.mean([moas(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    
    metrics.update({dataset_name:{
        'smape': smapes,
        'mase': mases,
        'mues': mueses,
        'moes': moeses,
        'muas': muases,
        'moas': moases
        
    }})
    
    counter += 1







                                      INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 16






  0%|          | 0/50 [00:00<?, ?it/s]GPU available: False, used: False
TPU available: False, using: 0 TPU cores
WARNING - (py.warnings._showwarnmsg) - /Users/polina/.conda/envs/neural_prophet/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: you defined a validation_step but have no val_dataloader. Skipping validation loop
  warnings.warn(*args, **kwargs)


  | Name      | Type         | Params
-------------------------------------------
0 | lstm      | LSTM         | 20.0 K
1 | linear    | Linear       | 195   
2 | loss_func | SmoothL1Loss | 0     
-------------------------------------------
20.2 K    Trainable params
0         Non-trainable params
20.2 K    Total params
0.081     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

WARNING - (py.warnings._showwarnmsg) - /Users/polina/.conda/envs/neural_prophet/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)



Training: 0it [00:00, ?it/s]

WARNING - (py.warnings._showwarnmsg) - /Users/polina/.conda/envs/neural_prophet/lib/python3.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()

WARNING - (py.warnings._showwarnmsg) - /Users/polina/.conda/envs/neural_prophet/lib/python3.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()

WARNING - (py.warnings._showwarnmsg) - /Users/polina/.conda/envs/neural_prophet/lib/python3.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()

WARNING - (py.warnings._showwarnmsg) - /Users/polina/.conda/envs/neural_prophet/lib/python3.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in double_scalars
  # This is added back by InteractiveShellA

...

Training: 0it [00:00, ?it/s]

INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 16




                                      
                                      

                                      


                                      GPU available: False, used: False
TPU available: False, using: 0 TPU cores
WARNING - (py.warnings._showwarnmsg) - /Users/polina/.conda/envs/neural_prophet/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: you defined a validation_step but have no val_dataloader. Skipping validation loop
  warnings.warn(*args, **kwargs)


  | Name      | Type         | Params
-------------------------------------------
0 | lstm      | LSTM         | 17.2 K
1 | linear    | Linear       | 195   
2 | loss_func | SmoothL1Loss | 0     
-------------------------------------------
17.3 K    Trainable params
0         Non-trainable params
17.3 K    Total params
0.069     Total estimated model params size (MB)


Validation sanity check: 0it [00:00, ?it/s]

WARNING - (py.warnings._showwarnmsg) - /Users/polina/.conda/envs/neural_prophet/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)



Training: 0it [00:00, ?it/s]

In [34]:
pd.DataFrame(metrics).to_csv('lstm_libra_fin.csv', index = True)

In [35]:
pd.DataFrame(metrics)

,finance_33.csv,finance_27.csv,finance_26.csv,finance_32.csv,finance_24.csv,finance_30.csv,finance_18.csv,finance_19.csv,finance_31.csv,finance_25.csv,...,finance_11.csv,finance_39.csv,finance_38.csv,finance_10.csv,finance_28.csv,finance_14.csv,finance_15.csv,finance_29.csv,finance_17.csv,finance_16.csv
smape,0.748306,1.860831,2.504210,3.745232,3.556806,1.653503,7.379438,4.455019,1.258408,1.973542,...,12.792803,8.789966,8.052506,9.120429,2.329199,9.866039,4.736665,1.915829,7.800125,5.294421
mase,inf,2.413253,5.024394,5.088500,inf,1.073399,2.836589,2.009576,3.709146,1.529645,...,3.055637,0.805823,0.539749,1.230603,2.023913,2.559182,1.348744,2.375485,2.027867,1.490802
mues,0.639344,0.803409,0.888228,0.939891,0.883792,0.814208,0.000000,0.142857,0.639344,0.551984,...,0.888889,0.786667,0.928571,0.222222,0.680162,1.000000,0.722222,0.340426,0.833333,0.666667
moes,0.360656,0.196591,0.111772,0.060109,0.116208,0.185792,1.000000,0.857143,0.360656,0.448016,...,0.111111,0.213333,0.071429,0.777778,0.319838,0.000000,0.277778,0.659574,0.166667,0.333333
muas,0.004857,0.016921,0.021587,0.037087,0.033485,0.016246,0.000000,0.006407,0.009913,0.012522,...,0.117061,0.092376,0.080543,0.014734,0.018837,0.093593,0.046809,0.008421,0.072139,0.049054
moas,0.003771,0.003771,0.009976,0.000622,0.002632,0.004777,0.077992,0.048852,0.004268,0.010574,...,0.026323,0.016641,0.002274,0.109072,0.007958,0.000000,0.017548,0.015556,0.010677,0.010799


In [37]:
from neuralprophet.utils.df_utils import split_df

metrics = {}

counter = 0
for dataset_name, df in datasets.items():
#     if counter == 1:
#         break
    
    idx_ts = int(dataset_name.split('.')[0].split('_')[-1])-1
    freq_number = frequencies.iloc[
        idx_ts][[col for col in frequencies.columns if usecase in col][0]]
    freq = mapping(frequencies.iloc[
        idx_ts][[col for col in frequencies.columns if usecase in col][0]])
    
    n_lags = freq_number
    n_forecasts = 3
    
    m = NeuralProphet(n_lags = n_lags,
            n_forecasts=n_forecasts,
            num_hidden_layers=1,
            d_hidden=64,
            learning_rate=0.1,
            epochs=50,
            batch_size=None,
            loss_func="Huber",
            optimizer="AdamW",
            train_speed=None,
            normalize="auto",
            impute_missing=True,)
    
    tr, vl = split_df(df, n_lags = n_lags, n_forecasts = n_forecasts, valid_p = valid_p)
    m.fit(tr, freq = freq)
    future = m.make_future_dataframe(vl, periods = 0, n_historic_predictions=True)
    forecast = m.predict(future)
    fold = forecast.iloc[n_lags:][[f'yhat{i}' for i in range(1, n_forecasts+1)]]

    y_predicted = [np.array(fold).diagonal(offset=-i) for i in range(len(fold) - n_forecasts + 1)]
    y = np.array(vl[n_lags:]["y"])
    y_rolled = [y[i : i + n_forecasts] for i in range(len(y) - n_forecasts + 1)]
    
    y_naive = np.array(vl[n_lags-1:-1]["y"])
    y_naive_rolled = [y_naive[i : i + n_forecasts] for i in range(len(y_naive) - n_forecasts + 1)]
    
    smapes = np.mean([smape(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    mases = np.mean([mase(y_rolled[i], y_predicted[i], y_naive_rolled[i]) for i in range(len(y_rolled))])
    mueses = np.mean([mues(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    moeses = np.mean([moes(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    muases = np.mean([muas(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    moases = np.mean([moas(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    
    metrics.update({dataset_name:{
        'smape': smapes,
        'mase': mases,
        'mues': mueses,
        'moes': moeses,
        'muas': muases,
        'moas': moases
        
    }})
    
    counter += 1

INFO - (NP.utils.set_auto_seasonalities) - Disabling weekly seasonality. Run NeuralProphet with weekly_seasonality=True to override this.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 16



  0%|          | 0/50 [00:00<?, ?it/s]GPU available: False, used: False
TPU available: False, using: 0 TPU cores
WARNING - (py.warnings._showwarnmsg) - /Users/polina/.conda/envs/neural_prophet/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: you defined a validation_step but have no val_dataloader. Skipping validation loop
  warnings.warn(*args, **kwargs)


  | Name          | Type          | Params
------------------------------------------------
0 | season_params | ParameterDict | 12    
1 | ar_net        | ModuleList    | 1.0 K 
2 | loss_func     | SmoothL1Loss  | 0     
-----------------------------------

Validation sanity check: 0it [00:00, ?it/s]

WARNING - (py.warnings._showwarnmsg) - /Users/polina/.conda/envs/neural_prophet/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)



Training: 0it [00:00, ?it/s]

WARNING - (py.warnings._showwarnmsg) - /Users/polina/.conda/envs/neural_prophet/lib/python3.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()

WARNING - (py.warnings._showwarnmsg) - /Users/polina/.conda/envs/neural_prophet/lib/python3.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()

WARNING - (py.warnings._showwarnmsg) - /Users/polina/.conda/envs/neural_prophet/lib/python3.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in double_scalars
  # This is added back by InteractiveShellApp.init_path()

WARNING - (py.warnings._showwarnmsg) - /Users/polina/.conda/envs/neural_prophet/lib/python3.7/site-packages/ipykernel_launcher.py:11: RuntimeWarning: divide by zero encountered in double_scalars
  # This is added back by InteractiveShellA

...

Training: 0it [00:00, ?it/s]

INFO - (NP.utils.set_auto_seasonalities) - Disabling yearly seasonality. Run NeuralProphet with yearly_seasonality=True to override this.
INFO - (NP.utils.set_auto_seasonalities) - Disabling daily seasonality. Run NeuralProphet with daily_seasonality=True to override this.
INFO - (NP.config.set_auto_batch_epoch) - Auto-set batch_size to 16






  0%|          | 0/50 [00:00<?, ?it/s]GPU available: False, used: False
TPU available: False, using: 0 TPU cores
WARNING - (py.warnings._showwarnmsg) - /Users/polina/.conda/envs/neural_prophet/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: you defined a validation_step but have no val_dataloader. Skipping validation loop
  warnings.warn(*args, **kwargs)


  | Name          | Type          | Params
------------------------------------------------
0 | season_params | ParameterDict | 6     
1 | ar_net        | ModuleList    | 320   
2 | loss_func     | SmoothL1Loss  | 0     
--------------------------------

Validation sanity check: 0it [00:00, ?it/s]

WARNING - (py.warnings._showwarnmsg) - /Users/polina/.conda/envs/neural_prophet/lib/python3.7/site-packages/pytorch_lightning/utilities/distributed.py:68: UserWarning: The dataloader, train dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  warnings.warn(*args, **kwargs)



Training: 0it [00:00, ?it/s]

In [38]:
pd.DataFrame(metrics).to_csv('np_libra_fin.csv', index = True)

In [39]:
pd.DataFrame(metrics)

,finance_33.csv,finance_27.csv,finance_26.csv,finance_32.csv,finance_24.csv,finance_30.csv,finance_18.csv,finance_19.csv,finance_31.csv,finance_25.csv,...,finance_11.csv,finance_39.csv,finance_38.csv,finance_10.csv,finance_28.csv,finance_14.csv,finance_15.csv,finance_29.csv,finance_17.csv,finance_16.csv
smape,1.768175,1.785110,4.839014,4.430679,13.943318,2.024036,12.391023,11.585891,6.617646,4.756577,...,15.908761,6.388955,64.770169,42.508554,3.492355,13.705860,9.806658,3.120847,6.812100,7.053095
mase,inf,2.256139,9.230062,6.938760,inf,1.408215,4.176157,7.139150,19.561150,5.030666,...,3.876151,0.622159,3.229358,7.226101,3.297267,3.442897,2.734479,4.114496,2.133255,2.546827
mues,0.573770,0.754261,0.597305,0.726776,0.660550,0.273224,0.611111,0.428571,0.000000,0.897619,...,0.111111,0.413333,1.000000,0.000000,0.454341,1.000000,0.777778,0.489362,0.500000,0.380952
moes,0.426230,0.245739,0.402695,0.273224,0.339450,0.726776,0.388889,0.571429,1.000000,0.102381,...,0.888889,0.586667,0.000000,1.000000,0.545659,0.000000,0.222222,0.510638,0.500000,0.619048
muas,0.015969,0.015640,0.025364,0.035776,0.116515,0.006616,0.111431,0.035637,0.000000,0.045126,...,0.025817,0.044158,0.478118,0.000000,0.016284,0.127987,0.087980,0.028830,0.034229,0.025453
moas,0.008354,0.004380,0.042424,0.010064,0.009188,0.023172,0.092309,0.131852,0.069028,0.003157,...,0.186843,0.046245,0.000000,0.555603,0.027008,0.000000,0.012200,0.031659,0.043585,0.075104


In [ ]:
from neuralprophet.utils.df_utils import split_df

metrics = {}

counter = 0
for dataset_name, df in datasets.items():
#     if counter == 1:
#         break
    
    idx_ts = int(dataset_name.split('.')[0].split('_')[-1])-1
    freq_number = frequencies.iloc[
        idx_ts][[col for col in frequencies.columns if usecase in col][0]]
    freq = mapping(frequencies.iloc[
        idx_ts][[col for col in frequencies.columns if usecase in col][0]])
    
    n_lags = freq_number
    n_forecasts = 3
    
    m = NBeats(context_length = int(n_lags), prediction_length = 3)
    
    tr, vl = split_df(df, n_lags = n_lags, n_forecasts = n_forecasts, valid_p = valid_p)
    m.fit(tr, freq = freq)
    future = m.make_future_dataframe(vl, freq = freq, periods = 1, n_historic_predictions=len(vl))
    forecast = m.predict(future)
    fold = forecast.iloc[n_lags:][[f'yhat{i}' for i in range(1, n_forecasts+1)]]

    y_predicted = [np.array(fold).diagonal(offset=-i) for i in range(len(fold) - n_forecasts + 1)]
    y = np.array(vl[n_lags:]["y"])
    y_rolled = [y[i : i + n_forecasts] for i in range(len(y) - n_forecasts + 1)]
    
    y_naive = np.array(vl[n_lags-1:-1]["y"])
    y_naive_rolled = [y_naive[i : i + n_forecasts] for i in range(len(y_naive) - n_forecasts + 1)]
    
    smapes = np.mean([smape(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    mases = np.mean([mase(y_rolled[i], y_predicted[i], y_naive_rolled[i]) for i in range(len(y_rolled))])
    mueses = np.mean([mues(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    moeses = np.mean([moes(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    muases = np.mean([muas(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    moases = np.mean([moas(y_rolled[i], y_predicted[i]) for i in range(len(y_rolled))])
    
    metrics.update({dataset_name:{
        'smape': smapes,
        'mase': mases,
        'mues': mueses,
        'moes': moeses,
        'muas': muases,
        'moas': moases
        
    }})
    
    counter += 1

GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name            | Type       | Params
-----------------------------------------------
0 | loss            | MASE       | 0     
1 | logging_metrics | ModuleList | 0     
2 | net_blocks      | ModuleList | 1.6 M 
-----------------------------------------------
1.6 M     Trainable params
0         Non-trainable params
1.6 M     Total params
6.434     Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Insufficient data for 529 historic forecasts, reduced to 169.
Number of forecast steps is defined by n_forecasts. Adjusted to 3.
WARNING - (py.warnings._showwarnmsg) - /Users/polina/.conda/envs/neural_prophet/lib/python3.7/site-packages/pandas/core/indexing.py:1738: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value[:, i].tolist(), pi)

GPU available: False, used: False
TPU available: False, using: 0 TPU cores

  | Name            | Type       | Params
-----------------------------------------------
0 | loss            | MASE       | 0     
1 | logging_metrics | ModuleList | 0     
2 | net_blocks      | ModuleList | 4.9 M 
-----------------------------------------------
4.9 M     Trainable params
0         Non-tr

Training: 0it [00:00, ?it/s]

In [ ]:
pd.DataFrame(metrics).to_csv('nbeats_libra_fin.csv', index = True)

In [16]:
metrics

{'economics_80.csv': {'smape': 14.227959482889146,
  'mase': 0.6793891006101452,
  'mues': 0.6845238095238094,
  'moes': 0.3154761904761904,
  'muas': 0.10425659722191864,
  'moas': 0.2198020913237889},
 'economics_94.csv': {'smape': 0.7190360801962151,
  'mase': 3.1163183418894147,
  'mues': 0.36507936507936506,
  'moes': 0.6349206349206349,
  'muas': 0.001503754856503776,
  'moas': 0.006099359626964676},
 'economics_43.csv': {'smape': 5.829709637480135,
  'mase': 1.0738051638414605,
  'mues': 0.725,
  'moes': 0.275,
  'muas': 0.05530059414642341,
  'moas': 0.028149931598778904},
 'economics_57.csv': {'smape': 0.1298987500371631,
  'mase': 0.4951778316399436,
  'mues': 0.1733333333333333,
  'moes': 0.8266666666666667,
  'muas': 0.0003500760304027347,
  'moas': 0.0014781010942533108},
 'economics_56.csv': {'smape': 3.0017473773169367,
  'mase': 2.069085805064148,
  'mues': 0.4533333333333333,
  'moes': 0.5466666666666666,
  'muas': 0.018881944477294157,
  'moas': 0.020638514320898648},

In [231]:
np.array(y_rolled).shape

(112, 3)

In [234]:
np.array(y_predicted).shape

(112, 3)

In [209]:
datasets.items()

dict_items([('economics_80.csv',               y         ds
0    118.546043 2000-01-01
1    104.810656 2000-01-02
2     86.660324 2000-01-03
3    100.003271 2000-01-04
4     95.392105 2000-01-05
..          ...        ...
926  110.599140 2002-07-15
927  152.393959 2002-07-16
928  124.138307 2002-07-17
929  101.671139 2002-07-18
930   94.411006 2002-07-19

[931 rows x 2 columns]), ('economics_94.csv',              y         ds
0    16.855022 2000-01-31
1    16.855022 2000-02-29
2    16.907295 2000-03-31
3    16.959569 2000-04-30
4    17.011842 2000-05-31
..         ...        ...
117  17.203512 2009-10-31
118  17.255786 2009-11-30
119  17.273210 2009-12-31
120  17.273210 2010-01-31
121  17.325484 2010-02-28

[122 rows x 2 columns]), ('economics_43.csv',                  y         ds
0    112607.239182 2000-01-31
1    104416.948874 2000-02-29
2    122801.816481 2000-03-31
3    108323.609959 2000-04-30
4    134909.726251 2000-05-31
..             ...        ...
415  349798.002672 2034-08-

In [207]:
idx_ts

98

In [186]:
n_lags

12

In [187]:
freq_number

12

In [188]:
m.make_future_dataframe(vl, n_historic_predictions = True)

,ds,y,t,y_scaled
0,2002-01-31,72.434388,1.044286,0.475743
1,2002-02-28,75.966344,1.084286,0.503912
2,2002-03-31,125.119406,1.128571,0.895931
3,2002-04-30,113.051888,1.171429,0.799687
4,2002-05-31,57.914122,1.215714,0.359937
5,2002-06-30,120.312021,1.258571,0.857590
6,2002-07-31,NaN,1.302857,NaN


In [189]:
vl

,y,ds
0,86.071665,2002-01-06
1,87.445203,2002-01-07
2,83.717027,2002-01-08
3,96.961864,2002-01-09
4,60.366870,2002-01-10
...,...,...
190,110.599140,2002-07-15
191,152.393959,2002-07-16
192,124.138307,2002-07-17
193,101.671139,2002-07-18


In [190]:
df

,y,ds
0,118.546043,2000-01-01
1,104.810656,2000-01-02
2,86.660324,2000-01-03
3,100.003271,2000-01-04
4,95.392105,2000-01-05
...,...,...
926,110.599140,2002-07-15
927,152.393959,2002-07-16
928,124.138307,2002-07-17
929,101.671139,2002-07-18


In [191]:
freq

'M'

In [147]:
len(df)

931

In [148]:
len(tr)

748

In [151]:
len(vl) - freq_number

183